In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import pickle
import json

In [29]:
df = pd.read_csv("diabetes_data_upload.csv")
df.sample(10)

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
507,60,Male,No,No,Yes,No,No,No,No,No,No,No,No,No,No,Yes,Negative
490,46,Male,No,No,No,Yes,No,No,No,Yes,No,Yes,No,No,Yes,No,Negative
375,38,Male,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Negative
313,54,Male,No,No,Yes,Yes,No,Yes,No,No,No,Yes,No,No,Yes,No,Negative
250,48,Female,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,Yes,No,No,No,Positive
408,59,Female,No,No,No,Yes,No,No,Yes,Yes,No,Yes,No,Yes,Yes,Yes,Negative
192,64,Male,No,Yes,No,No,No,No,No,No,Yes,Yes,No,No,No,No,Positive
396,30,Male,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Negative
382,68,Male,Yes,Yes,No,No,Yes,No,Yes,Yes,Yes,No,Yes,No,No,No,Positive
142,49,Male,Yes,Yes,Yes,No,No,No,No,Yes,No,No,No,No,No,No,Positive


In [40]:
df.iloc[313]

Age                         54
Gender                    Male
Polyuria                    No
Polydipsia                  No
sudden weight loss         Yes
weakness                   Yes
Polyphagia                  No
Genital thrush             Yes
visual blurring             No
Itching                     No
Irritability                No
delayed healing            Yes
partial paresis             No
muscle stiffness            No
Alopecia                   Yes
Obesity                     No
class                 Negative
Name: 313, dtype: object

In [30]:
df.columns

Index(['Age', 'Gender', 'Polyuria', 'Polydipsia', 'sudden weight loss',
       'weakness', 'Polyphagia', 'Genital thrush', 'visual blurring',
       'Itching', 'Irritability', 'delayed healing', 'partial paresis',
       'muscle stiffness', 'Alopecia', 'Obesity', 'class'],
      dtype='object')

In [31]:
X = df.drop('class', axis=1)
y = df['class']

In [32]:
X['Gender'].replace(['Male', 'Female'], [1, 0], inplace=True)
X['Polyuria'].replace(['Yes', 'No'], [1, 0], inplace=True)
X['Polydipsia'].replace(['Yes', 'No'], [1, 0], inplace=True)
X['sudden weight loss'].replace(['Yes', 'No'], [1, 0], inplace=True)
X['weakness'].replace(['Yes', 'No'], [1, 0], inplace=True)
X['Polyphagia'].replace(['Yes', 'No'], [1, 0], inplace=True)
X['Genital thrush'].replace(['Yes', 'No'], [1, 0], inplace=True)
X['visual blurring'].replace(['Yes', 'No'], [1, 0], inplace=True)
X['Itching'].replace(['Yes', 'No'], [1, 0], inplace=True)
X['Irritability'].replace(['Yes', 'No'], [1, 0], inplace=True)
X['delayed healing'].replace(['Yes', 'No'], [1, 0], inplace=True)
X['partial paresis'].replace(['Yes', 'No'], [1, 0], inplace=True)
X['muscle stiffness'].replace(['Yes', 'No'], [1, 0], inplace=True)
X['Alopecia'].replace(['Yes', 'No'], [1, 0], inplace=True)
X['Obesity'].replace(['Yes', 'No'], [1, 0], inplace=True)


In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True,random_state=4, stratify=y)

In [34]:
y_train

469    Negative
66     Positive
494    Negative
506    Negative
312    Negative
         ...   
518    Negative
411    Negative
507    Negative
49     Positive
359    Positive
Name: class, Length: 468, dtype: object

In [35]:
clf = RandomForestClassifier()
clf.fit(X_train.values, y_train)
y_pred_train = clf.predict(X_train.values)
y_pred = clf.predict(X_test.values)
y_acc = accuracy_score(y_test, y_pred)
y_acc_train = accuracy_score(y_train, y_pred_train)
print(y_acc_train)
print(y_acc)

1.0
1.0


In [36]:
import pickle
pickle.dump(clf, open('model.pkl', 'wb'))

In [37]:
model = pickle.load(open('model.pkl', 'rb'))

In [41]:
print(model.predict([[54, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0]]))

['Negative']


In [ ]:
54
Male
No
No
 Yes
weakness                   Yes
Polyphagia                  No
Genital thrush             Yes
visual blurring             No
Itching                     No
Irritability                No
delayed healing            Yes
partial paresis             No
muscle stiffness            No
Alopecia                   Yes
Obesity                     No
class                 Negative